In [3]:
import torch
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as fun

First I need to import the dataset, and to do this I need to format the data a little 
src: Udacity, June 2021, mnist_mlp_exercise.ipynb, source code, https://www.udacity.com/course/intro-to-machine-learning-nanodegree--nd229 
    NOTE: The code cited above was left partially blank and was also filled in with my own work, but as I can no longer differentiate what I worked on I am citing the whole file
dataset source: Cohen, G., Afshar, S., Tapson, J., & van Schaik, A. (2017). EMNIST: an extension of MNIST to handwritten letters. Retrieved from http://arxiv.org/abs/1702.05373
    NOTE: This is the official citation for the EMNIST dataset, though it's call is built into torch

In [4]:
batch = 20
#Creates a transform for the data that will make a tensor of the image data
tensorform = transforms.ToTensor()
#Sets up the groups of data
trainingData = datasets.EMNIST(root='dataset', train='True', split='balanced', download='True', transform=tensorform)
testingData = datasets.EMNIST(root='dataset', train='False', split='balanced', download='True', transform=tensorform)
#Loads them to be used for training purposes
trainingLoad = torch.utils.data.DataLoader(trainingData, batch_size=batch, num_workers=0, shuffle=True, drop_last=True)
testingLoad = torch.utils.data.DataLoader(testingData, batch_size=batch, num_workers=0, shuffle=True, drop_last=True)

import matplotlib.pyplot as plt
%matplotlib inline
    
# obtain one batch of training images
dataiter = iter(trainingLoad)
images, labels = dataiter.next()
images = images.numpy()

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    # print out the correct label for each image
    # .item() gets the value contained in a Tensor
    ax.set_title(str(labels[idx].item()))

Designes the neural network that will be used to train our ml model.
Since I will be needing an output of 26 letters, for both capital and lower case, and 10 digits, I will need a total of 62 outputs, if I decrease the linear level by a factor of 2 accross 4 layers, we will get a starting number of layers of 992, followed by 496, 248, and  124.

In [16]:
# class myNN(nn.Module):
#     def __init__(self):
#         super(myNN, self).__init__()
#         self.forward1 = nn.Linear(784, 512)
#         self.forward2 = nn.Linear(512, 256)
#         self.forward3 = nn.Linear(256, 256)
#         self.forward35 = nn.Linear(256, 128)
#         self.forward4 = nn.Linear(128, 47)
#         #sets a dropout rate for the individual nodes of 15%
#         self.dropout = nn.Dropout(p=.15)

#     def forward(self, element):
#         element = element.view(-1, 784)
#         element = self.dropout(fun.relu(self.forward1(element)))
#         element = self.dropout(fun.relu(self.forward2(element)))
#         element = self.dropout(fun.relu(self.forward3(element)))
#         element = self.dropout(fun.relu(self.forward35(element)))
#         element = self.forward4(element)
#         return element
from MyNN import myNN

Next I will need to initialize the neural network model and choose the loss function that will be used, as well as choose the optimization function

In [17]:
model = myNN()
lossfun = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.03)

Actually training the network:
    I need a loop that goes through a number of iterations that will be defined beforehand 
    Need to track the efficiency of the training, so I will need to also test each iteration
    

In [18]:
totalIterations = 10
#set the model to training mode
model.train()

for iteration in range(totalIterations):
    train_loss = 0.0

    for data, target in trainingLoad:
        optimizer.zero_grad()
        output = model(data)
        #figure out the loss
        loss = lossfun(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
    train_loss = train_loss/len(trainingLoad.dataset)

    #prints the information about how well training is going
    print("Iteration ", iteration, "\tTraining loss: {:.6f}".format(train_loss))
        

Iteration  0 	Training loss: 2.314045
Iteration  1 	Training loss: 0.876970
Iteration  2 	Training loss: 0.688836
Iteration  3 	Training loss: 0.599636
Iteration  4 	Training loss: 0.547879
Iteration  5 	Training loss: 0.508455
Iteration  6 	Training loss: 0.479222
Iteration  7 	Training loss: 0.456705
Iteration  8 	Training loss: 0.437283
Iteration  9 	Training loss: 0.419537


for __, data in trainingData:
    print(data)

Tests the formerly trained model using the test set that was created initially 

In [15]:
tloss = 0.0
groupCorrect = list(0. for i in range(62))
groupTotal = list(0. for i in range(62))
#set the model to evaluation mode so that it doesn't try to alter it, also speeds up the testing
model.eval()

for data, target in testingLoad:
    out = model(data)
    loss = lossfun(out, target)
    #update the test's loss
    tloss += loss.item() * data.size(0)
    #converts the output to a prediction
    _, prediction = torch.max(out, 1)
    correct = np.squeeze(prediction.eq(target.data.view_as(prediction)))
    for i in range(batch):
        label = target.data[i] 
        groupCorrect[label]  += correct[i].item()
        groupTotal[label] += 1
#I anticipate some higher loss in testing as the model will not be used to this dataset
tloss = tloss/len(testingLoad.dataset)
print('Test loss: {:.6f}\n'.format(tloss))

for i in range(62):
    if groupTotal[i] > 0:
        print("Test accuracy of\t", i, ": ", (100 * groupCorrect[i] / groupTotal[i]), "% (", groupCorrect[i], "/", groupTotal[i], ")")

Test loss: 0.192353

Test accuracy of	 0 :  74.79166666666667 % ( 1795.0 / 2400.0 )
Test accuracy of	 1 :  81.29166666666667 % ( 1951.0 / 2400.0 )
Test accuracy of	 2 :  98.5 % ( 2364.0 / 2400.0 )
Test accuracy of	 3 :  99.45833333333333 % ( 2387.0 / 2400.0 )
Test accuracy of	 4 :  98.20833333333333 % ( 2357.0 / 2400.0 )
Test accuracy of	 5 :  91.33333333333333 % ( 2192.0 / 2400.0 )
Test accuracy of	 6 :  97.20833333333333 % ( 2333.0 / 2400.0 )
Test accuracy of	 7 :  99.5 % ( 2388.0 / 2400.0 )
Test accuracy of	 8 :  98.66666666666667 % ( 2368.0 / 2400.0 )
Test accuracy of	 9 :  91.58333333333333 % ( 2198.0 / 2400.0 )
Test accuracy of	 10 :  99.54166666666667 % ( 2389.0 / 2400.0 )
Test accuracy of	 11 :  98.04166666666667 % ( 2353.0 / 2400.0 )
Test accuracy of	 12 :  98.95833333333333 % ( 2375.0 / 2400.0 )
Test accuracy of	 13 :  97.33333333333333 % ( 2336.0 / 2400.0 )
Test accuracy of	 14 :  99.54166666666667 % ( 2389.0 / 2400.0 )
Test accuracy of	 15 :  65.75 % ( 1578.0 / 2400.0 )
Tes

The next section is for saving the model for use in other scripts

In [10]:
torch.save(model, 'nnmodel.pth')